In [1]:
import cv2
import numpy as np
import math

In [2]:
model = cv2.imread("reference/image.jpg", 1)
model = cv2.imread("reference/santo_patrono.jpg", 1)
model = cv2.imread("reference/scaled_rolling_school.png", 1)
stream = cv2.imread("reference/ref_smol.jpg", 1)

In [3]:
"""
SIFT - SURF - ORB
"""
model = cv2.imread("reference/image.jpg", 1)

# ORB keypoint detector
orb = cv2.ORB_create(nfeatures=1500)
# Compute model keypoints and its descriptors
kp_model, des_model = orb.detectAndCompute(model, None)

frame = cv2.drawKeypoints(model, kp_model, None)
cv2.imshow("model", frame)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [6]:
"""
Keypoint in frame
"""

stream = cv2.imread("reference/ref_smol.jpg", 1)

# ORB keypoint detector
orb = cv2.ORB_create(nfeatures=1500)
# Compute model keypoints and its descriptors
# find and draw the keypoints of the frame
kp_frame, des_frame = orb.detectAndCompute(frame, None)

frame = cv2.drawKeypoints(stream, kp_frame, None)
cv2.imshow("model", frame)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
"""
Try to find the ref
Aruco Marker
"""
model = cv2.imread("reference/image.jpg", 1)
frame = cv2.imread("reference/ref_smol.jpg", 1)

# ORB keypoint detector
orb = cv2.ORB_create(nfeatures=1500)
# Compute model keypoints and its descriptors
kp_model, des_model = orb.detectAndCompute(model, None)

# find and draw the keypoints of the frame
kp_frame, des_frame = orb.detectAndCompute(frame, None)

# BFMatcher object based on hamming distance
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
# match frame descriptors with model descriptors
matches = bf.match(des_model, des_frame)

# sort them in the order of their distance
# the lower the distance, the better the match
matches = sorted(matches, key=lambda x: x.distance)

frame = cv2.drawMatches(
            model, kp_model, frame, kp_frame, matches[:100], 0, flags=2
        )


cv2.imshow("model", frame)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [8]:
"""
Santo patrono
"""
model = cv2.imread("reference/santo_patrono.jpg", 1)
frame = cv2.imread("reference/ref_smol.jpg", 1)

# ORB keypoint detector
orb = cv2.ORB_create(nfeatures=1500)
# Compute model keypoints and its descriptors
kp_model, des_model = orb.detectAndCompute(model, None)

# find and draw the keypoints of the frame
kp_frame, des_frame = orb.detectAndCompute(frame, None)

# BFMatcher object based on hamming distance
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
# match frame descriptors with model descriptors
matches = bf.match(des_model, des_frame)
# sort them in the order of their distance
# the lower the distance, the better the match
matches = sorted(matches, key=lambda x: x.distance)

frame = cv2.drawMatches(
            model, kp_model, frame, kp_frame, matches[:100], 0, flags=2
        )


cv2.imshow("model", frame)

cv2.waitKey(0)
cv2.destroyAllWindows()

Identificar key feachures con ORB BFMatcher y RANSAC<br>
Matriz de homografia para rotrotransladar el objeto

In [9]:
def projection_matrix(camera_parameters, homography):
    """
    From the camera calibration matrix and the estimated homography
    compute the 3D projection matrix
    """
    # Compute rotation along the x and y axis as well as the translation
    homography = homography * (-1)
    rot_and_transl = np.dot(np.linalg.inv(camera_parameters), homography)
    col_1 = rot_and_transl[:, 0]
    col_2 = rot_and_transl[:, 1]
    col_3 = rot_and_transl[:, 2]
    # normalise vectors
    l = math.sqrt(np.linalg.norm(col_1, 2) * np.linalg.norm(col_2, 2))
    rot_1 = col_1 / l
    rot_2 = col_2 / l
    translation = col_3 / l
    # compute the orthonormal basis
    c = rot_1 + rot_2
    p = np.cross(rot_1, rot_2)
    d = np.cross(c, p)
    rot_1 = np.dot(
        c / np.linalg.norm(c, 2) + d / np.linalg.norm(d, 2), 1 / math.sqrt(2)
    )
    rot_2 = np.dot(
        c / np.linalg.norm(c, 2) - d / np.linalg.norm(d, 2), 1 / math.sqrt(2)
    )
    rot_3 = np.cross(rot_1, rot_2)
    # finally, compute the 3D projection matrix from the model to the current frame
    projection = np.stack((rot_1, rot_2, rot_3, translation)).T
    return np.dot(camera_parameters, projection)


Creemos un objeto y usemos la matriz homografica para ponerlo en el espacio

In [10]:
import cv2
import numpy as np
import math

# Minimum number of matches that have to be found
# to consider the recognition valid
MIN_MATCHES = 75

homography = None

# matrix of camera parameters
camera_parameters = np.array([[800, 0, 320], [0, 800, 240], [0, 0, 1]])

# ORB keypoint detector
orb = cv2.ORB_create()

# BFMatcher object based on hamming distance
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Reference surface
logo = cv2.imread('models/gdg_ramos_scaled.jpeg')
model = cv2.imread("reference/santo_patrono.jpg", 0)
h, w = model.shape

# Compute model keypoints and its descriptors
kp_model, des_model = orb.detectAndCompute(model, None)


# init video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    # find and draw the keypoints of the frame
    kp_frame, des_frame = orb.detectAndCompute(frame, None)
    # match frame descriptors with model descriptors
    matches = bf.match(des_model, des_frame)
    # sort them in the order of their distance
    # the lower the distance, the better the match
    matches = sorted(matches, key=lambda x: x.distance)
        
    # compute Homography if enough matches are found
    if len(matches) > MIN_MATCHES:
        # differenciate between source points and destination points
        src_pts = np.float32([kp_model[m.queryIdx].pt for m in matches]).reshape(
            -1, 1, 2
        )
        dst_pts = np.float32([kp_frame[m.trainIdx].pt for m in matches]).reshape(
            -1, 1, 2
        )
        # compute Homography
        homography, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)    
        
        # if a valid homography matrix was found render cube on model plane
        if homography is not None:

            # Drawing plane
            lh, lw, ch = logo.shape
            
            x_offset = int(w/2-lw/2)
            y_offset = 0
                
            pts = np.float32(
                [
                 [w/2-lw/2, 0], 
                 [w/2-lw/2, lh], 
                 [w/2+lw/2, lh], 
                 [w/2+lw/2, 0]
                ]

            ) 
            # project corners into frame
            dst = cv2.perspectiveTransform(pts.reshape(-1, 1, 2), homography)

            if (dst >= 0).all():
                
                color = np.array([42, 126, 32])
                color = list(color[::-1])
                color = [int(c) for c in color]
                imgpts = np.int32(dst[:,:2])
                
                #project background
                frame = cv2.fillConvexPoly(frame, imgpts, color)
                # connect them with lines
                frame = cv2.polylines(frame, [imgpts], True, 255, 3, cv2.LINE_AA)

        """
        except:
            pass
        """
        
        # show result
        cv2.imshow("frame", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
        
        
    else:
        print(f"Not enough matches found - {len(matches)}/{MIN_MATCHES}")    
        
cap.release()
cv2.destroyAllWindows()



### Agregar una imagen 2D a nuestro proyecto

Aca debemos aplicar una logica de mascaras para poder "sumar" las imagenes.<br/>
Luego usamos warpPerspective podemos darle el angulo que la matriz homografica nos define.

In [20]:
# logo = cv2.imread('models/pyar_logo_scaled.png')
# logo = cv2.imread('models/gdg_ramos_scaled.jpeg')
logo = cv2.imread("models/scaled_rolling_school.png")

cv2.imshow("model", logo)

cv2.waitKey(0)
cv2.destroyAllWindows()

Para poder sumar 2 imagenes deben tener el mismo tamaño para esto debemos crear una mascara del tamaño de nuestro frame

In [16]:
fh,fw,fc = frame.shape
lh, lw, ch = logo.shape
            
x_offset = int(fw/2-lw/2)
y_offset = int(fh/2-lh/2)

mask = np.ones([fh, fw, fc], dtype=np.uint8)

mask[y_offset:logo.shape[0]+y_offset,x_offset:logo.shape[1]+x_offset] = logo

cv2.imshow("model", mask)

cv2.waitKey(0)
cv2.destroyAllWindows()

Ahora aplicamos la matriz homografica a nuestra mascara y sumamos el resultado al frame.<br/>
Al utilizar una operación bit a bit tipo `or` todos los pixeles negros 0 se sumaran con cualquiero otro pixel de color
<br/>
<table>
    <tbody>
        <tr><th><p>Frame</p></th>
            <th><p>Mask</p></th>
            <th><p>Result</p></th>
        </tr>
        <tr>
            <td><p>0</p></td>
            <td><p>0</p></td>
            <td><p>0</p></td>
        </tr>
        <tr>
            <td><p>0</p></td>
            <td><p>n</p></td>
            <td><p>n</p></td>
        </tr>
        <tr>
            <td><p>m</p></td>
            <td><p>0</p></td>
            <td><p>m</p></td>
        </tr>
        <tr>
            <td><p>m</p></td>
            <td><p>n</p></td>
            <td><p>m+n</p></td>
        </tr>
    </tbody>
</table>
El maximo valor es 255 (blanco)

In [19]:
import cv2
import numpy as np
import math

# Minimum number of matches that have to be found
# to consider the recognition valid
MIN_MATCHES = 75

homography = None

# matrix of camera parameters
camera_parameters = np.array([[800, 0, 320], [0, 800, 240], [0, 0, 1]])

# ORB keypoint detector
orb = cv2.ORB_create()

# BFMatcher object based on hamming distance
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Reference surface
model = cv2.imread("reference/santo_patrono.jpg", 0)
h, w = model.shape

# Compute model keypoints and its descriptors
kp_model, des_model = orb.detectAndCompute(model, None)


# init video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    # find and draw the keypoints of the frame
    kp_frame, des_frame = orb.detectAndCompute(frame, None)
    # match frame descriptors with model descriptors
    matches = bf.match(des_model, des_frame)
    # sort them in the order of their distance
    # the lower the distance, the better the match
    matches = sorted(matches, key=lambda x: x.distance)
        
    # compute Homography if enough matches are found
    if len(matches) > MIN_MATCHES:
        # differenciate between source points and destination points
        src_pts = np.float32([kp_model[m.queryIdx].pt for m in matches]).reshape(
            -1, 1, 2
        )
        dst_pts = np.float32([kp_frame[m.trainIdx].pt for m in matches]).reshape(
            -1, 1, 2
        )
        # compute Homography
        homography, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)    
        
        # if a valid homography matrix was found render cube on model plane
        if homography is not None:

            # Drawing plane
            lh, lw, ch = logo.shape
            
            x_offset = int(w/2-lw/2)
            y_offset = int(h/2-lh/2)
                
            
            pts = np.float32(
                [
                 [w/2-lw/2, h/2-lh/2], 
                 [w/2-lw/2, h/2+lh/2], 
                 [w/2+lw/2, h/2+lh/2], 
                 [w/2+lw/2, h/2-lh/2]
                ]

            ) 

            # project corners into frame
            dst = cv2.perspectiveTransform(pts.reshape(-1, 1, 2), homography)

            if (dst >= 0).all():
                
                #Lets get the affine transformation
                
                
                fh,fw,fc = frame.shape
                mask = np.ones([fh, fw, fc], dtype=np.uint8)
                
                mask[y_offset:logo.shape[0]+y_offset,x_offset:logo.shape[1]+x_offset] = logo
                logo_warped = cv2.warpPerspective(mask, homography, (fw,fh))
                
                color = np.array([0, 0, 0])
                color = list(color[::-1])
                color = [int(c) for c in color]
                imgpts = np.int32(dst[:,:2])
                
                #project background
                frame = cv2.fillConvexPoly(frame, imgpts, color)
                # connect them with lines
                frame = cv2.polylines(frame, [imgpts], True, 255, 3, cv2.LINE_AA)
                # Project logo
                frame = cv2.bitwise_or(logo_warped, frame)
                cv2.imshow("mask", logo_warped)

        """
        except:
            pass
        """
        # draw first 100 matches.
        
        """
        frame = cv2.drawMatches(
            model, kp_model, frame, kp_frame, matches[:100], 0, flags=2
        )
        """
        
        # show result
        cv2.imshow("frame", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
        
        
    else:
        print(f"Not enough matches found - {len(matches)}/{MIN_MATCHES}")    
        
cap.release()
cv2.destroyAllWindows()



### Hora de jugar

In [21]:
import cv2
import numpy as np
import math
from pokemons.generator import random_pokemon_img

# Minimum number of matches that have to be found
# to consider the recognition valid
MIN_MATCHES = 75
SHOW_IMG_TIME = 30
show_timer = SHOW_IMG_TIME
random_timer = 0
RANDOM_TIMER_END = 30
img_path = 'pokemons/images'

homography = None

# matrix of camera parameters
camera_parameters = np.array([[800, 0, 320], [0, 800, 240], [0, 0, 1]])

# ORB keypoint detector
orb = cv2.ORB_create()

# BFMatcher object based on hamming distance
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Reference surface
model = cv2.imread("reference/santo_patrono.jpg", 0)
h, w = model.shape

# Compute model keypoints and its descriptors
kp_model, des_model = orb.detectAndCompute(model, None)


# init video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    # find and draw the keypoints of the frame
    kp_frame, des_frame = orb.detectAndCompute(frame, None)
    # match frame descriptors with model descriptors
    matches = bf.match(des_model, des_frame)
    # sort them in the order of their distance
    # the lower the distance, the better the match
    matches = sorted(matches, key=lambda x: x.distance)
        
    # compute Homography if enough matches are found
    if len(matches) > MIN_MATCHES:
        # differenciate between source points and destination points
        src_pts = np.float32([kp_model[m.queryIdx].pt for m in matches]).reshape(
            -1, 1, 2
        )
        dst_pts = np.float32([kp_frame[m.trainIdx].pt for m in matches]).reshape(
            -1, 1, 2
        )
        # compute Homography
        homography, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)    
        
        # if a valid homography matrix was found render cube on model plane
        if homography is not None:
            if random_timer > 0:
                random_timer -= 1
                new_img = random_pokemon_img(img_path)
                logo = cv2.imread(new_img)
                if show_timer > 0:
                    show_timer -= 1
                else:
                    show_timer = SHOW_IMG_TIME

            # Drawing plane
            lh, lw, ch = logo.shape
            
            x_offset = int(w/2-lw/2)
            y_offset = int(h/2-lh/2)
            
            pts = np.float32(
                [
                 [w/2-lw/2, h/2-lh/2], 
                 [w/2-lw/2, h/2+lh/2], 
                 [w/2+lw/2, h/2+lh/2], 
                 [w/2+lw/2, h/2-lh/2]
                ]

            ) 
            
            # project corners into frame
            dst = cv2.perspectiveTransform(pts.reshape(-1, 1, 2), homography)

            if (dst >= 0).all():
                fh,fw,fc = frame.shape
                mask = np.ones([fh, fw, fc], dtype=np.uint8)
                
                mask[y_offset:logo.shape[0]+y_offset,x_offset:logo.shape[1]+x_offset] = logo
                logo_warped = cv2.warpPerspective(mask, homography, (fw,fh))
                
                color = np.array([0, 0, 0])
                color = list(color[::-1])
                color = [int(c) for c in color]
                imgpts = np.int32(dst[:,:2])
                
                #project background
                frame = cv2.fillConvexPoly(frame, imgpts, color)

                # Project logo
                frame = cv2.bitwise_or(logo_warped, frame)

        """
        except:
            pass
        """
        # draw first 100 matches.
        
        """
        frame = cv2.drawMatches(
            model, kp_model, frame, kp_frame, matches[:100], 0, flags=2
        )
        """
        
        # show result
        cv2.imshow("frame", frame)
        key = cv2.waitKey(33)
        if key == ord("q"):
            break
        if key == ord("p"):
            random_timer = RANDOM_TIMER_END
        
        
    else:
        print(f"Not enough matches found - {len(matches)}/{MIN_MATCHES}")    
        
cap.release()
cv2.destroyAllWindows()



## Render 3D
Bueno pero que tan complejo será mostrar cualquier objeto 3D?

In [22]:
def projection_matrix(camera_parameters, homography):
    """
    From the camera calibration matrix and the estimated homography
    compute the 3D projection matrix
    """
    # Compute rotation along the x and y axis as well as the translation
    homography = homography * (-1)
    rot_and_transl = np.dot(np.linalg.inv(camera_parameters), homography)
    col_1 = rot_and_transl[:, 0]
    col_2 = rot_and_transl[:, 1]
    col_3 = rot_and_transl[:, 2]
    # normalise vectors
    l = math.sqrt(np.linalg.norm(col_1, 2) * np.linalg.norm(col_2, 2))
    rot_1 = col_1 / l
    rot_2 = col_2 / l
    translation = col_3 / l
    # compute the orthonormal basis
    c = rot_1 + rot_2
    p = np.cross(rot_1, rot_2)
    d = np.cross(c, p)
    rot_1 = np.dot(
        c / np.linalg.norm(c, 2) + d / np.linalg.norm(d, 2), 1 / math.sqrt(2)
    )
    rot_2 = np.dot(
        c / np.linalg.norm(c, 2) - d / np.linalg.norm(d, 2), 1 / math.sqrt(2)
    )
    rot_3 = np.cross(rot_1, rot_2)
    # finally, compute the 3D projection matrix from the model to the current frame
    projection = np.stack((rot_1, rot_2, rot_3, translation)).T
    return np.dot(camera_parameters, projection)

def render(img, obj, projection, model, color=False):
    """
    Render a loaded obj model into the current video frame
    """
    vertices = obj.vertices
    h, w = model.shape
     #Scale into view with scale matrix
    x_scale = 300
    y_scale = 300
    z_scale = 300
    scale_matrix = np.array([[x_scale, 0, 0],
                             [0, y_scale, 0],
                             [0, 0, z_scale]])

    for face in obj.faces:
        face_vertices = face[0]
        points = np.array([vertices[vertex - 1] for vertex in face_vertices])
        points = np.dot(points, scale_matrix)
        # render model in the middle of the reference surface. To do so,
        # model points must be displaced
        points = np.array([[p[0] + w / 2, p[1] + h / 2, p[2]] for p in points])
        dst = cv2.perspectiveTransform(points.reshape(-1, 1, 3), projection)
        imgpts = np.int32(dst)
        
        color = True
        if color is False:
            cv2.fillConvexPoly(img, imgpts, (137, 27, 211))
        else:
            # Get Normals and check if is visible
            line1 = points[1] - points[0]
            line2 = points[2] - points[0]

            normal = np.cross(line1,line2)
            norm_len = np.linalg.norm(normal)
            normal = (normal*1/norm_len)
            
            if np.dot(normal,points[0]) < 0:

                directional_light = np.array([0,0,-1000])
                light_len = np.linalg.norm(directional_light)
                light_normal = (directional_light*1/light_len)

                lum_val = np.dot(normal,light_normal)

                color = np.array([0, 204, 255]) * 1/(lum_val+0.01)
                color = list(color[::-1])
                color = [int(c) for c in color]

                cv2.fillConvexPoly(img, imgpts, color)

    return img

In [23]:
"""
Adding model
"""

import cv2
import numpy as np
import math

from objloader_simple import OBJ
fox = OBJ("models/fox.obj")

# Minimum number of matches that have to be found
# to consider the recognition valid
MIN_MATCHES = 75

homography = None

# matrix of camera parameters
camera_parameters = np.array([[800, 0, 320], [0, 800, 240], [0, 0, 1]])

# ORB keypoint detector
orb = cv2.ORB_create()

# BFMatcher object based on hamming distance
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Reference surface
model = cv2.imread("reference/santo_patrono.jpg", 0)
h, w = model.shape

# Compute model keypoints and its descriptors
kp_model, des_model = orb.detectAndCompute(model, None)


# init video capture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    # find and draw the keypoints of the frame
    kp_frame, des_frame = orb.detectAndCompute(frame, None)
    # match frame descriptors with model descriptors
    matches = bf.match(des_model, des_frame)
    # sort them in the order of their distance
    # the lower the distance, the better the match
    matches = sorted(matches, key=lambda x: x.distance)
        
    # compute Homography if enough matches are found
    if len(matches) > MIN_MATCHES:
        # differenciate between source points and destination points
        src_pts = np.float32([kp_model[m.queryIdx].pt for m in matches]).reshape(
            -1, 1, 2
        )
        dst_pts = np.float32([kp_frame[m.trainIdx].pt for m in matches]).reshape(
            -1, 1, 2
        )
        # compute Homography
        homography, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)    
        
        # if a valid homography matrix was found render cube on model plane
        if homography is not None:
            # obtain 3D projection matrix from homography matrix and camera parameters
            projection = projection_matrix(camera_parameters, homography)
            # project cube or model
            frame = render(frame, fox, projection, model, False)
        
        # show result
        cv2.imshow("frame", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
        
        
    else:
        print(f"Not enough matches found - {len(matches)}/{MIN_MATCHES}")    
        
cap.release()
cv2.destroyAllWindows()



In [ ]:
cap.release()
cv2.destroyAllWindows()